In [1]:
import tensorflow as tf
from pathlib import Path

In [4]:
url = "https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
path = tf.keras.utils.get_file("spa-eng.zip", origin=url, cache_dir="datasets/", extract=True)

extracted_path = Path(path).with_name("spa-eng") / "spa.txt"
text = extracted_path.read_text(encoding="utf-8")

In [5]:
import numpy as np

text = text.replace("¡", "").replace("¿", "")
pairs = [line.split("\t") for line in text.splitlines()]
np.random.seed(42) 
np.random.shuffle(pairs)

sentences_en, sentences_es = zip(*pairs)

for i in range(3):
    print(sentences_en[i], "=>", sentences_es[i])

How boring! => Qué aburrimiento!
I love sports. => Adoro el deporte.
Would you like to swap jobs? => Te gustaría que intercambiemos los trabajos?


In [6]:
vocab_size = 1000
max_length = 50

text_vec_layer_en = tf.keras.layers.TextVectorization(vocab_size, output_sequence_length=max_length)
text_vec_layer_es = tf.keras.layers.TextVectorization(vocab_size, output_sequence_length=max_length)

text_vec_layer_en.adapt(sentences_en)
text_vec_layer_es.adapt([f"startofseq {s} endofseq" for s in sentences_es])

X_train = tf.constant(sentences_en[:100_000])
X_valid = tf.constant(sentences_en[100_000:])

X_train_dec = tf.constant([f"startofseq {s}" for s in sentences_es[:100_000]])
X_valid_dec = tf.constant(["startofseq {s}" for s in sentences_es[100_000:]])

y_train = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[:100_000]])
y_valid = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[100_000:]])

In [9]:
encoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
decoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)

embed_size = 128

encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_es(decoder_inputs)

encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True)
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True)

encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)

encoder = tf.keras.layers.LSTM(512, return_state=True)
encoder_outputs, *encoder_state = encoder(encoder_embeddings)

decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

#### Attention

In [7]:
tf.random.set_seed(42)
encoder = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256, return_sequences=True, return_state=True)
)

In [10]:
attention_layer = tf.keras.layers.Attention()
attention_outputs = attention_layer([decoder_outputs, encoder_outputs])

output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
y_proba = output_layer(attention_outputs)

In [13]:
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=[y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

history = model.fit((X_train, X_train_dec), y_train, epochs=5, validation_data=((X_valid, X_valid_dec), y_valid))

Epoch 1/5
3125/3125 [==============================] - 3063s 977ms/step - loss: 3.8432 - accuracy: 0.3090 - val_loss: 4.8245 - val_accuracy: 0.1940
Epoch 2/5
3125/3125 [==============================] - 3132s 1s/step - loss: 3.3852 - accuracy: 0.3548 - val_loss: 4.6441 - val_accuracy: 0.2342
Epoch 3/5
3125/3125 [==============================] - 3191s 1s/step - loss: 3.1816 - accuracy: 0.3885 - val_loss: 4.5730 - val_accuracy: 0.2747
Epoch 4/5
3125/3125 [==============================] - 3115s 997ms/step - loss: 3.0365 - accuracy: 0.4103 - val_loss: 4.4828 - val_accuracy: 0.2964
Epoch 5/5
3125/3125 [==============================] - 3122s 999ms/step - loss: 2.9128 - accuracy: 0.4266 - val_loss: 4.4368 - val_accuracy: 0.3172


In [14]:
def translate(sentence_en):
    translation = ""
    for word_inx in range(max_length):
        X = np.array([sentence_en])
        X_dec = np.array(["startofseq" + translation])
        
        y_proba = model.predict((X, X_dec), verbose=0)[0, word_inx]
        
        predicted_word_id = np.argmax(y_proba)
        predicted_word = text_vec_layer_es.get_vocabulary()[predicted_word_id]
        
        if predicted_word == "endofseq":
            break
        translation += " " + predicted_word
        
    return translation

In [33]:
translate("I like soccer and also going to the beach")

''

#### Attention Is All You Need: The Transformer Architecture
#### Positional encodings

In [18]:
max_length = 50
embed_size = 128

pos_embed_layer = tf.keras.layers.Embedding(max_length, embed_size)
batch_max_len_enc = tf.shape(encoder_embeddings)[1]

encoder_in = encoder_embeddings + pos_embed_layer(tf.range(batch_max_len_enc))
batch_max_len_dec = tf.shape(decoder_embeddings)[1]

decoder_in = decoder_embeddings + pos_embed_layer(tf.range(batch_max_len_dec))

In [28]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, max_length, embed_size, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        assert embed_size % 2 == 0, "embed_size must be even"
        
        p, i = np.meshgrid(np.arange(max_length), 2 * np.arange(embed_size // 2))
        
        pos_emb = np.empty((1, max_length, embed_size))
        pos_emb[0, :, ::2] = np.sin(p / 10_000 ** (i / embed_size)).T
        pos_emb[0, :, 1::2] = np.cos(p / 10_000 ** (i / embed_size)).T
        
        self.pos_encodings = tf.constant(pos_emb.astype(self.dtype))
        self.supports_masking = True

    def call(self, inputs):
        batch_max_length = tf.shape(inputs)[1]
        return inputs + self.pos_encodings[:, :batch_max_length]

In [30]:
pos_embed_layer = PositionalEncoding(max_length, embed_size)
encoder_in = pos_embed_layer(encoder_embeddings)
decoder_in = pos_embed_layer(decoder_embeddings)

#### Multi-Head Attention

In [31]:
N = 2 # instead of 6 since we don not have huge training set
num_heads = 8
dropout_rate = 0.1
n_units = 128
encoder_pad_mask = tf.math.not_equal(encoder_input_ids, 0)[:, tf.newaxis]
Z = encoder_in

for _ in range(N):
    skip = Z
    attn_layer = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    
    Z = attn_layer(Z, value=Z, attention_mask=encoder_pad_mask)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
    
    Z = tf.keras.layers.Dense(n_units, activation="relu")(Z)
    Z = tf.keras.layers.Dense(embed_size)(Z)
    
    Z = tf.keras.layers.Dropout(dropout_rate)(Z)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))

In [36]:
decoder_pad_mask = tf.math.not_equal(decoder_input_ids, 0)[:, tf.newaxis]
causal_mask = tf.linalg.band_part(tf.ones((batch_max_len_dec, batch_max_len_dec), tf.bool), -1, 0)

In [37]:
encoder_outputs = Z  # let's save the encoder's final outputs
Z = decoder_in  # the decoder starts with its own inputs
for _ in range(N):
    skip = Z
    attn_layer = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    
    Z = attn_layer(Z, value=Z, attention_mask=causal_mask & decoder_pad_mask)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
    
    skip = Z
    attn_layer = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=encoder_outputs, attention_mask=encoder_pad_mask)
    
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
    skip = Z
    
    Z = tf.keras.layers.Dense(n_units, activation="relu")(Z)
    Z = tf.keras.layers.Dense(embed_size)(Z)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))

In [39]:
y_proba = tf.keras.layers.Dense(vocab_size, activation="softmax")(Z)

model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=[y_proba])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",metrics=["accuracy"])
history = model.fit((X_train, X_train_dec), y_train, epochs=5,validation_data=((X_valid, X_valid_dec), y_valid))

Epoch 1/5
3125/3125 [==============================] - 1923s 612ms/step - loss: 0.9751 - accuracy: 0.8390 - val_loss: 10.2249 - val_accuracy: 0.0524
Epoch 2/5
3125/3125 [==============================] - 1879s 601ms/step - loss: 0.6278 - accuracy: 0.8867 - val_loss: 11.0345 - val_accuracy: 0.0524
Epoch 3/5
3125/3125 [==============================] - 1851s 592ms/step - loss: 0.6063 - accuracy: 0.8879 - val_loss: 11.5364 - val_accuracy: 0.0524
Epoch 4/5
3125/3125 [==============================] - 1891s 605ms/step - loss: 0.5924 - accuracy: 0.8884 - val_loss: 10.5728 - val_accuracy: 0.0524
Epoch 5/5
3125/3125 [==============================] - 1896s 607ms/step - loss: 0.5745 - accuracy: 0.8900 - val_loss: 11.8870 - val_accuracy: 0.0524


In [40]:
translate("I like soccer and also going to the beach")

' [UNK]'